# Mastering Group Operations with Pandas: A Deep Dive

Welcome to this hands-on tutorial on one of the most powerful features in the pandas library: **group operations**. In the world of data science, your work will almost always involve analyzing and comparing different segments of your data. Whether you're comparing sales across different regions, user behavior between different demographics, or medical outcomes between treatment groups, the ability to effectively group and analyze data is a fundamental skill.

### The Power of Split-Apply-Combine

The mental model we will use for these operations is called **Split-Apply-Combine**. It's a simple but powerful three-step process:

1.  **Split:** We take a larger dataset and break it down into smaller, logical chunks based on the values in one or more columns (our "keys"). Think of this as sorting a deck of cards into separate piles for each suit (Hearts, Diamonds, Clubs, Spades).
2.  **Apply:** We then apply a function to each of these smaller chunks independently. This could be anything from a simple aggregation (like calculating the average) to a complex custom function.
3.  **Combine:** Finally, the results from applying the function to each chunk are intelligently combined back into a single, clean output object (usually a pandas Series or DataFrame).

### Our Case Study: The Titanic Dataset

Throughout this notebook, we will use the famous **Titanic dataset**. This dataset contains information about the passengers aboard the Titanic, including whether they survived, their age, their passenger class, and more. It's an excellent dataset for learning because it's easy to understand and contains a rich mix of data types (categorical, numerical, text), making it perfect for asking interesting questions that can be answered with group operations.

## 1. Setup and Data Exploration

First, let's import the pandas library and load our dataset directly from a public URL. We'll then take a quick look at its structure to understand what we're working with.

In [ ]:
import pandas as pd
import numpy as np

# The Titanic dataset is publicly available from the seaborn library's GitHub repository
url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv'
df = pd.read_csv(url)

print("First 5 rows of the Titanic dataset:")
df.head()

In [ ]:
print("A concise summary of the DataFrame:")
df.info()

From `.info()`, we can see we have columns of different types (`int64`, `float64`, `object`). We also see that the `age`, `embarked`, and `deck` columns have missing values, which will be interesting to handle later.

---

## 2. The `groupby()` Operation: The "Split" Step

The journey begins with the `.groupby()` method. When you call `df.groupby('some_column')`, pandas doesn't immediately compute anything. Instead, it creates a special `DataFrameGroupBy` object.

**What is this object?** Think of it as a blueprint or a set of instructions. It contains all the information pandas needs to efficiently process the data once you tell it *what* to do. It has already conceptually split the data into groups behind the scenes, and now it's just waiting for your command.

Let's see this in action.

In [ ]:
# Group the DataFrame by the 'sex' column
grouped_by_sex = df.groupby('sex')

# This doesn't output data, just tells you what kind of object you have
print(grouped_by_sex)

### Peeking Inside the Groups

While the `GroupBy` object itself isn't very descriptive, we can iterate through it to see the actual groups that were formed. Each iteration gives us a tuple containing the **group name** and the **DataFrame** of data belonging to that group.

In [ ]:
# Iterating through the groups to see the actual data chunks
for name, group_df in grouped_by_sex:
    print(f"--- Group: {name} ---")
    print(f"This group has {len(group_df)} rows.")
    print(group_df.head(3))
    print("\n")

### Grouping by Multiple Columns

You can group by more than one column to create more granular chunks. Just pass a list of column names. When you do this, the `name` of each group will be a tuple.

In [ ]:
# Grouping by both passenger class ('pclass') and sex
grouped_multi = df.groupby(['pclass', 'sex'])

# Let's look at the first few groups
for name, group_df in list(grouped_multi)[:3]:
    print(f"--- Group: {name} ---")
    print(f"This group has {len(group_df)} rows.")
    print(group_df.head(2))
    print("\n")

### Selecting a Single Group

If you want to pull out the DataFrame for just one specific group, you can use the `.get_group()` method.

In [ ]:
# Getting the DataFrame for only the male passengers in 1st class
first_class_males = grouped_multi.get_group((1, 'male'))
first_class_males.head()

### ✍️ Exercises for Section 2

1.  Create a `GroupBy` object by grouping the Titanic data by the `'embarked'` column, which indicates the port where a passenger boarded.
2.  Iterate through the groups you just created and print the number of passengers that embarked from each port.
3.  Group the data by both `'pclass'` and `'survived'`. Use `.get_group()` to select and display the first 5 rows of data for passengers who were in 3rd class and did not survive (survived=0).

---

## 3. Aggregation: The "Apply" and "Combine" Steps

Now that we know how to split the data, let's do something useful with the groups. **Aggregation** is the process of applying a function to each group that summarizes the data into a single value. This is how we answer questions like:

-   What was the *average age* of passengers in each class?
-   How *many people* survived vs. did not survive?
-   What was the *maximum fare* paid by men vs. women?

After creating a `GroupBy` object, you can select a column and apply an aggregation function like `.mean()`, `.sum()`, `.count()`, `.max()`, etc.

In [ ]:
# What was the average age for male and female passengers?
# Split by 'sex', apply 'mean()' to the 'age' column, and combine.
avg_age_by_sex = df.groupby('sex')['age'].mean()

print(avg_age_by_sex)

The result is a pandas Series where the index is our group key (`'sex'`) and the values are the results of our aggregation (`mean` of `age`).

### Applying Multiple Aggregation Functions

What if you want to calculate multiple statistics at once? The `.agg()` method is your best friend. You can pass it a list of function names.

In [ ]:
# For each passenger class, let's get multiple stats on the 'fare' paid
fare_stats_by_class = df.groupby('pclass')['fare'].agg(['mean', 'median', 'std', 'count', 'max'])

print(fare_stats_by_class)

### Applying Different Functions to Different Columns

The real power of `.agg()` shines when you want to apply different functions to different columns. You can do this by passing a dictionary where the keys are the column names and the values are the aggregation functions you want to apply.

In [ ]:
# For each passenger class, let's find:
# 1. The average fare
# 2. The median age
# 3. The total number of survivors

summary_by_class = df.groupby('pclass').agg(
    {
        'fare': 'mean',      # Calculate the mean of the 'fare' column
        'age': 'median',     # Calculate the median of the 'age' column
        'survived': 'sum'    # Calculate the sum of the 'survived' column
    }
)

print(summary_by_class)

### ✍️ Exercises for Section 3

1.  Calculate the overall **survival rate** for each passenger class (`pclass`). (Hint: Since the `survived` column is `1` for survivors and `0` otherwise, the `mean()` of this column is the survival rate!).
2.  Find the average `fare` paid by passengers from each embarkation port (`'embarked'`).
3.  Using a single `.agg()` call, find the following for each group in the `'sex'` column:
    *   The total number of survivors (`sum` of `'survived'`)
    *   The average `age`
    *   The maximum `fare` paid

---

## 4. `apply()`: Your Tool for Complex, Custom Operations

While aggregation with functions like `mean()` and `sum()` is common, it's often not enough. What if you want to perform a more complex operation for each group? For example:

-   For each passenger class, who was the oldest passenger?
-   For each port of embarkation, what was the survival rate of only the adult passengers?

Simple aggregation can't answer these. This is where the `.apply()` method comes in. It is the most flexible of the group operation methods. You can pass it a custom function, and this function will be executed on each group's DataFrame.

The function you write for `.apply()` takes a DataFrame (a single group) as input and can return a Series, a DataFrame, or even a single value.

In [ ]:
# Goal: For each passenger class, find the passenger who paid the highest fare.

# First, we define a function that will operate on each group's DataFrame.
# It finds the index of the max fare and returns the row.
def find_highest_fare_passenger(group_df):
    # Find the index of the row with the maximum fare in this group
    idx_of_max = group_df['fare'].idxmax()
    # Return the entire row for that passenger
    return group_df.loc[idx_of_max]

# Now, we split by 'pclass' and apply our custom function
highest_fare_passengers = df.groupby('pclass').apply(find_highest_fare_passenger)

print(highest_fare_passengers[['name', 'fare', 'age']])

This is incredibly powerful! We've written a custom piece of logic and applied it to each segment of our data, with pandas handling the splitting and combining for us.

### Using `apply` to Fill Missing Values

A very common use case in data science is to fill missing values (`NaN`s) with a meaningful statistic. It's often better to fill missing ages with the *average age of the group* rather than the overall average age. For instance, the average age of a 1st class passenger might be very different from a 3rd class passenger. We can do this with `.apply()` and a `lambda` function.

In [ ]:
# First, let's see the average age in each class
print("Average age per class:")
print(df.groupby('pclass')['age'].mean())

# Now, let's define a function to fill NaN with the group's mean age
fill_group_mean = lambda group: group.fillna(group.mean())

# We group by class, select the age column, and apply our function
df['age_filled'] = df.groupby('pclass')['age'].apply(fill_group_mean)

# Let's check our work. We'll look at a few rows where age was originally NaN
print("\nOriginal vs. Filled Age:")
print(df[df['age'].isna()][['pclass', 'age', 'age_filled']].head())

Notice how the `age_filled` for the 3rd class passenger is ~24.1, and for the 1st class passenger, it's ~38.2, matching the group averages. This is a much more nuanced way to handle missing data. (Note: We will explore an even better way to do this in the next section!).

### ✍️ Exercises for Section 4

1.  For each passenger `sex` ('male' and 'female'), find the record of the **oldest** passenger.
2.  Create a function that calculates the **survival rate** for a group. Apply this function to find the survival rate for passengers who embarked at each port (`'embarked'`).
3.  Group the data by `'pclass'`. Use `.apply()` along with the `.describe()` method to get a full statistical summary of the `fare` for each passenger class. The output should be a DataFrame showing the count, mean, std, etc., for the fare in each class.

---

## 5. `transform()`: Broadcasting Results Back to the Original Shape

We've seen `.agg()` which returns one result per group, and `.apply()` which can return almost anything. Now we introduce `.transform()`, which has a very specific and useful behavior: **it returns an object that has the exact same size and index as the original DataFrame or Series.**

**Why is this useful?** Its main purpose is to "broadcast" a group-level calculation to every row in that group. This is perfect for feature engineering, where you want to create a new column in your original DataFrame based on a group property.

Let's compare `mean()` with `transform('mean')` to understand the difference.

In [ ]:
# .mean() returns one value per group (a Series with 3 rows)
agg_result = df.groupby('pclass')['age'].mean()
print("Result of .mean() aggregation:")
print(agg_result)
print(f"Length of result: {len(agg_result)}\n")

# .transform('mean') returns a value for every row in the original data (a Series with 891 rows)
transform_result = df.groupby('pclass')['age'].transform('mean')
print("Result of .transform('mean'):")
print(transform_result.head())
print(f"Length of result: {len(transform_result)}")

The `transform` result is aligned with our original DataFrame's index, making it trivial to create a new column.

In [ ]:
# Create a new column showing the average age for each passenger's class
df['class_average_age'] = df.groupby('pclass')['age'].transform('mean')

# Display the new column alongside the original age and class
df[['pclass', 'age', 'class_average_age']].head()

### A Better Way to Fill Missing Values

Remember how we used `.apply()` to fill missing `age` values? `.transform()` provides a much cleaner and often more efficient way to do the exact same thing.

In [ ]:
# Fill missing age values using the median age of the passenger's class
# This is a one-liner and is highly readable!
df['age_filled_transform'] = df['age'].fillna(df.groupby('pclass')['age'].transform('median'))

# Let's verify that it worked and that there are no more NaNs
print(f"Number of missing ages after transform fill: {df['age_filled_transform'].isna().sum()}")
df[['pclass', 'age', 'age_filled_transform']].head(10)

### Group-wise Normalization for Feature Engineering

Another common data science task is to normalize a feature (like `fare`) within a specific category. For example, a fare of $30 might be very high for a 3rd class passenger but low for a 1st class passenger. We can create a new feature that shows how far each passenger's fare is from the average of their class.

This is a classic use case for `transform`.

In [ ]:
# Create a new column 'fare_deviation_from_class_mean'
mean_fare_by_class = df.groupby('pclass')['fare'].transform('mean')
df['fare_deviation'] = df['fare'] - mean_fare_by_class

df[['name', 'pclass', 'fare', 'fare_deviation']].head()

### ✍️ Exercises for Section 5

1.  Create a new column in the `df` DataFrame called `'fare_by_class'` that contains the *average* fare for each passenger's class (`pclass`).
2.  Fill the missing values in the `'embarked'` column using the *mode* (the most frequent value) of the passenger's class (`pclass`). (Hint: `lambda group: group.mode()[0]` might be helpful within a transform).
3.  Create a new column called `'age_rank_in_class'` that shows the rank of each passenger's age within their `pclass` (e.g., the oldest person in 1st class would have a rank of 1.0). (Hint: use `.transform('rank', ascending=False)`).

---

## 6. Conclusion and Recap

Congratulations! You have just worked through the most important grouping functionalities in pandas. Understanding how and when to use these tools is a crucial step in becoming a proficient data analyst or data scientist.

Let's quickly recap the tools we've learned:

| Method | What It Does | When to Use It | Output Shape |
| :--- | :--- | :--- | :--- |
| **`.agg()`** | Aggregates each group down to a single value (or a few values). | When you need summary statistics for each group (e.g., mean, median, count, sum). | One row per group. |
| **`.apply()`** | Applies a flexible, custom function to each group's DataFrame. | For complex, group-specific operations that can't be done with simple aggregation (e.g., selecting top N rows, group-wise regression). | Can be anything: a value, Series, or DataFrame. |
| **`.transform()`** | Applies a function to each group and broadcasts the result back to the original shape. | For feature engineering: when you need to create a new column in your original DataFrame based on a group-level property (e.g., group mean, group-wise rank). | Same shape as the original input. |

Mastering the split-apply-combine pattern will enable you to perform sophisticated, segmented analysis with just a few lines of code.